# Evolutionary transition rates of leaf endopolyploidy

Written by James Seery (jseery@mail.uoguelph.ca)

## Setup

#### Load packages and functions

In [ ]:
tryCatch(    # Load corHMM package. Provides the functions for ancestral reconstruction and inference of transition rates.
    library(corHMM),
    error = install.packages("corHMM", repos = "http://cran.utstat.utoronto.ca/")
)
tryCatch(    # Load CAPER package. Provides the ability to prune a large phylogeny.
    library(caper),
    error = install.packages("caper", repos = "http://cran.utstat.utoronto.ca/")
)
source(corHMM_functions.R)

#### Import dataset: entire Zanne phylogeny (phylo); and species data (endo)

In [ ]:
phylo = read.tree("Zanne.nwk")
phylo = makeLabels(phylo) # Add unique labels to all unnamed nodes
endo = read.csv("EI.csv") # Species names that match the tree in column 1 (labeled 'name', and leaf endoreduplication index in column 2 (labeled 'state').

#### Turn endopolyploidy into a character with two states

In [ ]:
threshold = 0.1 # Specify threshold endoreduplication index for deciding whether a species has endopolyploid leaves.
endo$state = state_threshold(endo$state, threshold)

#### Match the phylogeny (~30,000 species) to the dataset (~168 species)

In [ ]:
data = comparative.data(phy = phylo,
                        data = endo,
                        names.col = "name")

## Run evolutionary rates models

In [ ]:
ARD_ancestorNonEndo = transition_model(data, 
                                       model = "ARD", 
                                       root_nonendo = TRUE)
ARD_ancestorUnknown = transition_model(data, 
                                       model = "ARD")
ARD_ancestorNonEndo$AIC - ARD_ancestorUnknown/